My plan is 

// function 1  generate
1. generate 100 chars, save them in sketch.js

// function 2   check loop
2. use chrome driver and selenium to run the sketch website

3. find body, check it's attribute

// function 3   delete one line.
4. if there is a "js error", delete one line.

5. do it repeately, until there is no error.

6. call function 1 again

7. call function 3 again

so, function 2 will call function 1 & 3.

no error -- 1

error --3


right now:

## Questions:

- [ ] file.open with "w" flag will make the file empty   wb


- [ ] The generated code looks terrible, can I integrate Reinforcement Learning so it can produce non-error codes?


- [ ] how to keep tf session alive as much as possible, so It's won't need to load model every time. will this it be faster?   

  - [ ] is my solution correct?
  
- [ ] if I call `generate()` then call `checkError()`, will I encounter some async issue?




In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import os
    

host_url = 'http://127.0.0.1:8000/'

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1024x800")
    

chrome_driver_path = os.path.join(os.getcwd(), "chromedriver")
driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver_path)

def getJSError():
    r = false
    driver.get(host_url)
    body = driver.find_element_by_tag_name("body")
    if body.get_attribute("jserror") == 'true':
        r = true
    driver.close()
    return r
          

<selenium.webdriver.remote.webelement.WebElement (session="3b78d8c8ae298f7987f160a5beb31340", element="0.4685503330610292-1")>
true


In [2]:
from __future__ import print_function

from six.moves import cPickle

from six import text_type


import tensorflow as tf
from model import Model

import os

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__
    

def parseCharArray(charArray):
    r = []
    string = ''
    for char in charArray:
        if char != '\n':
            string += char
        else:
            r.append(string)
            string = ''
    return r
    

host_url = 'http://127.0.0.1:8000/'

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1024x800")

args = dotdict({'n':1000, 'save_dir': os.path.curdir + '/model/', 'prime':'', 'sample':1 })

model_dir = os.path.curdir + '/model/'

sketch_path = './p5template/sketch.js'
    

chrome_driver_path = os.path.join(os.getcwd(), "chromedriver")
driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver_path)

def getJSError():
    r = False
    driver.get(host_url)
    body = driver.find_element_by_tag_name("body")
    if body.get_attribute("jserror") == 'true':
        r = True
    driver.close()
    return r
          

def generate(num):
    newLines = parseCharArray(model.sample(sess, chars, vocab, num, args.prime,
                       args.sample))
    content.extend(newLines)
    print(content)

    file = open(sketch_path, "w")
    file.write("\n".join(content))
    file.close()

def removeOne():
    with open(sketch_path) as sketch:
        content = sketch.read().splitlines() 

    content.pop()
    file = open(sketch_path, "w")

    file.write("\n".join(content))
    file.close()

with open(sketch_path) as sketch:
    content = sketch.read().splitlines() 
with open(os.path.join(model_dir, 'config.pkl'), 'rb') as f:
    saved_args = cPickle.load(f)
with open(os.path.join(model_dir, 'chars_vocab.pkl'), 'rb') as f:
    chars, vocab = cPickle.load(f)
#Use most frequent char if no prime is given
if args.prime == '':
    args.prime = chars[0]
model = Model(saved_args, training=False)
# with tf.Session() as sess:
   

sess = tf.Session()
tf.global_variables_initializer().run()
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.get_checkpoint_state(args.save_dir)
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
    while True:
        if getJSError() == True:
            removeOne()
        else:
            generate(100)

sess.close()
        
 


ValueError: Variable rnnlm/softmax_w already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/Users/YANG/dev/myGithub/p5AI/model.py", line 48, in __init__
    [args.rnn_size, args.vocab_size])
  File "<ipython-input-1-5bf8648ce7ec>", line 91, in <module>
    model = Model(saved_args, training=False)
  File "/Users/YANG/.pyenv/versions/3.5.0/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
